In [1]:
import os
import warnings
import numpy as np
import luigi
import emu.neuralynx_io as nlx
from emu.pdil.raw import Electrophysiology,Participant,get_data_manifest,points_to_choice
from emu.neuralynx_io import nev_as_records
from emu.nwb import nlx_to_nwb
from emu.pipeline.remote import RemoteCSV
from pynwb import TimeSeries, NWBFile,NWBHDF5IO
from pynwb.ecephys import ElectricalSeries
from pynwb.misc import AnnotationSeries
import pandas as pd
import datetime
import glob
from emu.luigi.box import ReadableBoxFile,BoxTarget,BoxClient,file_id_to_path,path_to_root,obj_to_path,path_to_obj

In [2]:
box = BoxClient()

In [3]:
file_id_to_path(633265117086)

'/EMU/STUDY_PDil/PT_01/electrode_locations.csv'

In [4]:
from pathlib import Path
import pathlib

In [5]:
isinstance('/EMU/STUDY_PDil/PT_01/SEEG/processed/PO_Day_02.nwb', Path)

False

In [6]:
obj = path_to_obj(box.conn,'/EMU/STUDY_PDil/PT_01/SEEG/processed/PO_Day_02.nwb')
# box.exists('/EMU/STUDY_PDil/PT_01/SEEG/processed/PO_Day_02.nwb')

In [ ]:
box.conn.ge

In [8]:
box.isdir('/EMU/STUDY_PDil/PT_01/SEEG/processed')

True

In [20]:
folder = path_to_obj(box.conn,'/EMU/STUDY_PDil/PT_01/SEEG/processed')

In [27]:
src_path = '/home/elijahc/.emu/pdil/pt_01/SEEG/processed/PO_Day_02.mat'
file_size = os.path.getsize(src_path)
file_name = os.path.split(src_path)[-1]+'.test'
upload_session = folder.create_upload_session(file_size, file_name)

In [36]:
with open(src_path, 'rb') as file:
    print(type(file))
    print('part_size: ',upload_session.part_size)
    
    part= file.read(upload_session.part_size)
    
    print('read_part: ', sys.getsizeof(part))

<class '_io.BufferedReader'>
part_size:  33554432
read_part:  33554465


In [34]:
import sys



In [35]:
sys.getsizeof(part)

33554465

In [41]:
upload_session.total_parts

25

In [54]:
def iter_bytes(part_size,total_parts,src_path):
    with open(src_path, 'rb') as file:
        for i in np.arange(total_parts):
            yield (i, file.read(part_size))

In [55]:
from tqdm import tqdm

In [56]:
for i,part_bytes in tqdm(iter_bytes(upload_session.part_size,upload_session.total_parts,src_path)):
    offset = upload_session.part_size*i
    
    total_size=upload_session.part_size*upload_session.total_parts
    
    part = upload_session.upload_part_bytes(part_bytes, offset, total_size)

0it [00:00, ?it/s]"PUT https://upload.box.com/api/2.0/files/upload_sessions/63725C813F7B02E4E61BF937AE7FE3A5" 400 176
{'Date': 'Fri, 13 Mar 2020 20:06:44 GMT', 'Content-Type': 'application/json; charset=UTF-8', 'Content-Length': '176', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=31536000'}
{'code': '---ange',
 'message': "Complete length 838860800 in Content-Range header doesn't match "
            'session file size 805815134',
 'request_id': 'b221e2353e689337cc87c64593a61ab8'}

0it [00:07, ?it/s]


BoxAPIException: Message: Complete length 838860800 in Content-Range header doesn't match session file size 805815134
Status: 400
Code: invalid_range
Request ID: b221e2353e689337cc87c64593a61ab8
Headers: {'Date': 'Fri, 13 Mar 2020 20:06:44 GMT', 'Content-Type': 'application/json; charset=UTF-8', 'Content-Length': '176', 'Connection': 'keep-alive', 'Strict-Transport-Security': 'max-age=31536000'}
URL: https://upload.box.com/api/2.0/files/upload_sessions/63725C813F7B02E4E61BF937AE7FE3A5
Method: PUT
Context Info: None

In [47]:
[sys.getsizeof(part) for p in gen_byte]

[33554465,
 33554465,
 33554465,
 33554465,
 33554465,
 33554465,
 33554465,
 33554465,
 33554465,
 33554465,
 33554465,
 33554465,
 33554465,
 33554465,
 33554465,
 33554465,
 33554465,
 33554465,
 33554465,
 33554465,
 33554465,
 33554465,
 33554465,
 33554465,
 33554465]

In [40]:
for p in upload_session.get_parts():
    print(p)

In [30]:
upload_session.upload_part_bytes()

25

In [ ]:

resp = folder.create_upload_session()

In [21]:
folder.get_all_metadata()

In [10]:
box.upload('/EMU/STUDY_PDil/PT_01/SEEG/processed', src_path)

File larger than 200Mb, using chunked uploader


<Box File - 633760195823 (PO_Day_02.mat)>

In [11]:
nev_id = 632777789711
nev_fp = '/home/elijahc/.emu/pdil/pt_01/SEEG/raw/PO_Day_02.Events.nev'

In [12]:
from emu.neuralynx_io import read_header, read_records,parse_header,load_nev

In [13]:
box = BoxClient()
root_folder = box.folder(106535506845)
f = box.file(nev_id)


In [14]:
import boxsdk
boxsdk.object.folder.Folder

boxsdk.object.folder.Folder

In [15]:
type(path_to_obj(box.conn,'/EMU/STUDY_PDil/PT_01/SEEG').get())

boxsdk.object.folder.Folder

In [16]:
box.isdir('/EMU/STUDY_PDil/PT_01/SEEG/')

Dropbox paths should NOT have trailing slashes. This causes additional API calls
Consider modifying your calls to isdir, so that they don't use paths than end with '/'


True

In [17]:
results = box.conn.search().query('SEEG',limit=100,order='relevance')
[obj_to_path(r) for r in results]
# for r in results:
#     print([f.name for f in r.path_collection['entries']])
#     print('name:{}, id:{}'.format(r.name,r.id))

['/EMU/STUDY_PDil/PT_01/SEEG',
 '/EMU/STUDY_deepfakeANDtrickShots/trickShots/PT_01/SEEG',
 '/EMU/RAW_ephysData_testing/test_pt/sEEG',
 '//SEEG_SampleDATA',
 '/EMU/_patient_manifest.csv']

In [18]:
box.isdir('/EMU/STUDY_PDil/PT_01/SEEG/')

Dropbox paths should NOT have trailing slashes. This causes additional API calls
Consider modifying your calls to isdir, so that they don't use paths than end with '/'


True

In [19]:
path_to_root(fold)

NameError: name 'fold' is not defined